<a href="https://colab.research.google.com/github/ritwiks9635/QA-Bot-on-P-L-Data/blob/main/Question_Answering_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install pdfplumber
!pip install tabula-py
!pip install langchain
!pip install pinecone-client
!pip install streamlit
!pip install gradio
!pip install sentence-transformers
!pip install python-dotenv

In [2]:
%pip install -U -q "google-generativeai>=0.8.3"

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

Pine_API_KEY = os.getenv("Pine_API_KEY")
Gemini_API_KEY = os.getenv("Gemini_API_KEY")

## **Extract Data from PDF Documents**

In [6]:
import pdfplumber
from tabula import read_pdf
import pandas as pd
import string
import logging
logging.getLogger("tabula").setLevel(logging.ERROR)



# Extract plain text using pdfplumber
def extract_text_from_pdf(pdf_path):
    text_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages[1:]:
            text = page.extract_text()
            if text:
                text_data.append(text)
    return text_data


# Extract tables using Tabula with custom formatting
def extract_tables_with_tabula(pdf_path):
    tables = read_pdf(pdf_path, pages="all", multiple_tables=True, pandas_options={"header": 0})
    formatted_tables = []

    for table in tables:
        if len(table) <= 2:
            continue

        # Replace NaN with empty strings
        table = table.fillna("")

        # Remove punctuation from cell values
        table = table.map(lambda x: str(x).translate(str.maketrans("", "", string.punctuation)))

        # Join columns and rows into key-value pairs
        formatted_table = []
        headers = table.columns
        for _, row in table.iterrows():
            row_data = " | ".join(
                f"{headers[col_index]}: {str(cell)}" for col_index, cell in enumerate(row)
            )
            formatted_table.append(row_data)

        formatted_tables.append("\n".join(formatted_table))

    return formatted_tables


# Main pipeline to extract both text and formatted tables
def process_pdf(pdf_path):
    text_data = extract_text_from_pdf(pdf_path)

    formatted_tables = extract_tables_with_tabula(pdf_path)

    return text_data, formatted_tables



pdf_path = "Sample Financial Statement.pdf"

# Extract text and tables
text_output, table_output = process_pdf(pdf_path)

# Print the extracted text
print("Extracted Text:")
for page_num, page_text in enumerate(text_output, start=1):
    print(f"Page {page_num}:\n{page_text}\n")
    break

# Print the formatted tables
print("\nFormatted Tables:")
for table_num, table in enumerate(table_output, start=1):
    print(f"Table {table_num}:\n{table}\n")
    break

Extracted Text:
Page 1:
INFOSYS LIMITED AND SUBSIDIARIES
(In ₹ crore )
Condensed Consolidated Balance Sheets as at Note No. March 31, 2024 March 31, 2023
ASSETS
Non-current assets
Property, plant and equipment 2.2 12,370 13,346
Right-of-use assets 2.19 6,552 6,882
Capital work-in-progress 293 288
Goodwill 2.3 7,303 7,248
Other intangible assets 1,397 1,749
Financial assets
Investments 2.4 11,708 12,569
Loans 2.5 34 39
Other financial assets 2.6 3,105 2,798
Deferred tax assets (net) 454 1,245
Income tax assets (net) 3,045 6,453
Other non-current assets 2.9 2,121 2,318
Total non-current assets 48,382 54,935
Current assets
Financial assets
Investments 2.4 12,915 6,909
Trade receivables 2.7 30,193 25,424
Cash and cash equivalents 2.8 14,786 12,173
Loans 2.5 248 289
Other financial assets 2.6 12,085 11,604
Income tax assets (net) 6,397 6
Other current assets 2.9 12,808 14,476
Total current assets 89,432 70,881
Total assets 137,814 125,816
EQUITY AND LIABILITIES
Equity
Equity share capital 2

## **Preprocessing Data**

In [7]:
import re
from typing import List

# Preprocess text data to clean, normalize and join (text, table)
def improved_preprocess_text(text):
    # Step 1: Normalize spaces
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    # Step 2: Normalize financial terms
    text = text.replace("₹", "INR")
    text = text.replace("crore", "cr")
    text = text.replace("lakh", "lac")

    # Step 3: Retain table-like formatting by preserving line breaks
    lines = text.split(". ")
    cleaned_lines = []
    for line in lines:
        if re.search(r'\d', line):
            cleaned_lines.append(line.strip())
        else:
            cleaned_lines.append(line.strip())

    return "\n".join(cleaned_lines)

# Combine preprocessed text and table data
def combine_text_and_table_data(preprocessed_text: List[str], formatted_tables: List[str]):
    combined_output = []

    for page_num, page_text in enumerate(preprocessed_text, start=1):
        combined_output.append(f"Page {page_num}:")
        combined_output.append(page_text)

        # If tables exist for this page, append them
        if page_num <= len(formatted_tables):
            combined_output.append("Tables:")
            combined_output.append(formatted_tables[page_num - 1])

    return "\n\n".join(combined_output)

def preprocess_data(text_output, table_output):
    # Preprocess the text data
    preprocessed_text = [improved_preprocess_text(page_text) for page_text in text_output]

    # Combine preprocessed text and table data
    final_output = combine_text_and_table_data(preprocessed_text, table_output)

    pages = final_output.split("\n\nPage ")

    # Check "Page X:" is preserved
    pages = [f"Page {page}" if not page.startswith("Page") else page for page in pages]
    return pages

pages = preprocess_data(text_output, table_output)
# Print the final combined output
print("Final Combined Output:")
print(pages[0])

Final Combined Output:
Page 1:

INFOSYS LIMITED AND SUBSIDIARIES (In INR cr ) Condensed Consolidated Balance Sheets as at Note No
March 31, 2024 March 31, 2023 ASSETS Non-current assets Property, plant and equipment 2.2 12,370 13,346 Right-of-use assets 2.19 6,552 6,882 Capital work-in-progress 293 288 Goodwill 2.3 7,303 7,248 Other intangible assets 1,397 1,749 Financial assets Investments 2.4 11,708 12,569 Loans 2.5 34 39 Other financial assets 2.6 3,105 2,798 Deferred tax assets (net) 454 1,245 Income tax assets (net) 3,045 6,453 Other non-current assets 2.9 2,121 2,318 Total non-current assets 48,382 54,935 Current assets Financial assets Investments 2.4 12,915 6,909 Trade receivables 2.7 30,193 25,424 Cash and cash equivalents 2.8 14,786 12,173 Loans 2.5 248 289 Other financial assets 2.6 12,085 11,604 Income tax assets (net) 6,397 6 Other current assets 2.9 12,808 14,476 Total current assets 89,432 70,881 Total assets 137,814 125,816 EQUITY AND LIABILITIES Equity Equity share cap

## **Split the Data in Smaller Chunks**

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_with_langchain(text, chunk_size=8000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    chunks = text_splitter.split_text(text)

    return chunks

chunked_data = [chunk_with_langchain(page) for page in pages]
print(f"Chunks from Page 2:\n{chunked_data[0]}")

Chunks from Page 2:
['Page 1:\n\nINFOSYS LIMITED AND SUBSIDIARIES (In INR cr ) Condensed Consolidated Balance Sheets as at Note No\nMarch 31, 2024 March 31, 2023 ASSETS Non-current assets Property, plant and equipment 2.2 12,370 13,346 Right-of-use assets 2.19 6,552 6,882 Capital work-in-progress 293 288 Goodwill 2.3 7,303 7,248 Other intangible assets 1,397 1,749 Financial assets Investments 2.4 11,708 12,569 Loans 2.5 34 39 Other financial assets 2.6 3,105 2,798 Deferred tax assets (net) 454 1,245 Income tax assets (net) 3,045 6,453 Other non-current assets 2.9 2,121 2,318 Total non-current assets 48,382 54,935 Current assets Financial assets Investments 2.4 12,915 6,909 Trade receivables 2.7 30,193 25,424 Cash and cash equivalents 2.8 14,786 12,173 Loans 2.5 248 289 Other financial assets 2.6 12,085 11,604 Income tax assets (net) 6,397 6 Other current assets 2.9 12,808 14,476 Total current assets 89,432 70,881 Total assets 137,814 125,816 EQUITY AND LIABILITIES Equity Equity share c

## **Embedding Data using Sentence Transformer**

In [ ]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def generate_embeddings(chunked_data):
    embeddings = []
    for page_chunks in chunked_data:
        page_embeddings = embed_model.encode(page_chunks, convert_to_tensor=True)
        embeddings.append(page_embeddings)
    return embeddings


embeddings = generate_embeddings(chunked_data)

print(f"Embeddings for Page 2: {embeddings[0]}")

## **Store the Data in Vector Database**

In [17]:
# Create Pinecone index
index_name = 'qa-bot'

# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(Pine_API_KEY, environment='us-west1-gcp')

# Check if index exists
pc.create_index(
    name=index_name,
    dimension=embeddings[0].shape[1],
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) )
index = pc.Index(index_name)

In [18]:
# Upsert embeddings into Pinecone
def upsert_embeddings(embeddings, chunked_data):
    for page_num, page_embeddings in enumerate(embeddings):
        for chunk_num, embedding in enumerate(page_embeddings):
            metadata = {
                "page": page_num + 1,
                "chunk": chunk_num,
                "text": chunked_data[page_num][chunk_num],
            }
            # Generate unique ID for each embedding
            unique_id = f"page-{page_num+1}-chunk-{chunk_num}"
            embedding_list = embedding.tolist() if hasattr(embedding, 'tolist') else list(embedding)
            index.upsert([(unique_id, embedding_list, metadata)])

# Upsert data
upsert_embeddings(embeddings, chunked_data)

print("Embeddings stored in Pinecone!")

Embeddings stored in Pinecone!


## **RAG Pipeline**

In [19]:
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')

In [20]:
def retrieve_documents(query, top_k=5):
    # Generate query embedding
    query_embedding = retrieval_model.encode(query).tolist()

    # Search for the most relevant documents
    results = index.query(vector = query_embedding, top_k=top_k, include_metadata=True)

    # Extract the text from the metadata
    retrieved_texts = [match['metadata']['text'] for match in results['matches']]
    return retrieved_texts


query = "What is the gross profit for Q3 2024?"
retrieved_contexts = retrieve_documents(query)
print("Retrieved Contexts:")
for context in retrieved_contexts:
    print(context)
    break

Retrieved Contexts:
Page 39:

X20AO 2.24 FUNCTION WISE CLASSIFICATION OF CONDENSED CONSOLIDATED STATEMENT OF PROFIT AND LOSS (In INR cr) Particulars Note No
Three months ended March 31, Year ended March 31, 2024 2023 2024 2023 Revenue from operations 2.16 37,923 37,441 153,670 146,767 Cost of Sales 26,748 26,011 107,413 102,353 Gross profit 11,175 11,430 46,257 44,414 Operating expenses Selling and marketing expenses 1,735 1,659 6,973 6,249 General and administration expenses 1,819 1,894 7,537 7,260 Total operating expenses 3,554 3,553 14,510 13,509 Operating profit 7,621 7,877 31,747 30,905 Other income, net 2.17 2,729 671 4,711 2,701 Finance cost 110 82 470 284 Profit before tax 10,240 8,466 35,988 33,322 Tax expense: Current tax 2.15 1,173 2,260 8,390 9,287 Deferred tax 2.15 1,092 72 1,350 (73) Profit for the period 7,975 6,134 26,248 24,108 Other comprehensive income Items that will not be reclassified subsequently to profit or loss Remeasurement of the net defined benefit liabilit

## **Generative Model**

In [21]:
import google.generativeai as genai
from google.api_core import retry
genai.configure(api_key=Gemini_API_KEY)

In [22]:
gen_model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.3,
        top_p=1,
        max_output_tokens=200,
    ))

In [23]:
def generate_answer(query, contexts):
    prompt = (
        "You are an expert financial analyst specializing in interpreting financial data and metrics from tabular and textual formats. "
        "Your task is to analyze the provided financial contexts carefully and provide a clear, concise, and well-reasoned answer to the query. "
        "If the data is insufficient to answer the query, state explicitly what is missing and why the question cannot be fully answered.\n\n"
        "### Contexts:\n"
        + "\n".join([f"Context {i+1}:\n{context}" for i, context in enumerate(contexts)]) +
        "\n\n"
        "### Query:\n"
        + query +
        "\n\n"
        "### Instructions:\n"
        "1. Summarize any relevant financial data from the provided contexts.\n"
        "2. Use logical reasoning to answer the query based on the summarized data.\n"
        "3. If the data is insufficient, explain what information is missing and why it is needed.\n\n"
        "### Answer:"
    )

    retry_policy = {
        "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
    }
    response = gen_model.generate_content(
        prompt,
        request_options=retry_policy
    )
    return response.text

In [24]:
from IPython.display import display, Markdown

def generate_pipeline(query, top_k=5):
    # Step 1: Retrieve relevant contexts
    contexts = retrieve_documents(query, top_k=top_k)

    # Step 2: Generate an answer using the retrieved contexts
    answer = generate_answer(query, contexts)
    return answer


query = "How do the net income and operating expenses compare for Q1 2024?"
final_answer = generate_pipeline(query)
print("Final Answer:")
display(Markdown(final_answer))

Final Answer:


Based on Context 1 (Page 39), for the three months ended March 31, 2024 (Q1 2024), Infosys Limited reported:

* **Net Income (Profit for the period):** 7,975 INR cr
* **Operating Expenses:** 3,554 INR cr (This is the sum of selling and marketing expenses and general and administration expenses).

Therefore, in Q1 2024, Infosys's net income significantly exceeded its operating expenses (7,975 INR cr vs 3,554 INR cr).  The net income was more than double the operating expenses.


In [25]:
query_2 = "What is the gross profit for Q3 2024?"
answer = generate_pipeline(query_2)
print("Final Answer:")
display(Markdown(answer))

Final Answer:


The provided text (Context 1, Page 39) states that the gross profit for the three months ended March 31, 2024 (Q3 2024) was **INR 11,175 crore**.


In [26]:
query_3 = "What are the total expenses for Q2 2024?"
answer = generate_pipeline(query_3)
print("Final Answer:")
display(Markdown(answer))

Final Answer:


The provided text gives the total expenses for the year ended March 31, 2024 (which includes Q2 2024) but not specifically for Q2 2024 alone.  Context 3 shows total expenses for the year ended March 31, 2024, were INR 122,393 crore.  Context 2 provides a breakdown of expenses for the *three months* ended March 31, 2024 (which *might* be Q2, depending on the company's fiscal year), totaling INR 30,412 crore. However, without knowing Infosys' fiscal year, we cannot definitively say if this three-month period represents Q2.  Therefore, the question cannot be fully answered.  To answer the query accurately, we need to know:

1. **Infosys' fiscal year:**  The provided data shows figures for the year ended

In [27]:
query_4 = "Show the operating margin for the past 6 months."
answer = generate_pipeline(query_4)
print("Final Answer:")
display(Markdown(answer))

Final Answer:


The provided data is insufficient to calculate the operating margin for the past six months.  While Context 1 and Context 4 provide data on revenue and operating expenses for the three months ended March 31, 2024 and the year ended March 31, 2024,  they do *not* contain data for the three months *prior* to March 31, 2024.  To calculate the operating margin for the past six months, we need revenue and operating expense figures for both the three months ended March 31, 2024 and the three months ended December 31, 2023.  Only with this complete data can we calculate the operating profit (Revenue - Operating Expenses) for each period and then divide by revenue to obtain the operating margin for each three-month period and subsequently calculate the six-month operating margin.


In [28]:
query_5 = "What is the total revenue for the year"
answer = generate_pipeline(query_5)
print("Final Answer:")
display(Markdown(answer))

Final Answer:


The total revenue for the year ended March 31, 2024 is INR 153,670 crore.  This information is explicitly stated in Context 1 and Context 2.


## **Gradio for UI**

In [32]:
import gradio as gr

def qa_bot_interface(pdf_file, query):
    """
    Handles user interaction with the QA bot.

    Args:
    - pdf_file (UploadedFile): The uploaded PDF file.
    - query (str): The financial query entered by the user.

    Returns:
    - str: Retrieved data (contexts).
    - str: Generated answer.
    """
    if pdf_file is None:
        return "No PDF uploaded.", "Please upload a PDF to start."

    if not query:
        return "No query provided.", "Please enter a query to get an answer."

    # Step 1: Extract text and table data from the PDF
    text_data, table_data = process_pdf(pdf_file.name)

    # Step 2: Preprocess the extracted data
    preprocessed_data = preprocess_data(text_data, table_data)

    # Step 3: Chunk the data
    chunked_data = [chunk_with_langchain(page) for page in preprocessed_data]

    # Step 4: Generate and store embeddings
    embeddings = generate_embeddings(chunked_data)
    upsert_embeddings(embeddings, chunked_data)

    # Step 5: Retrieve relevant documents and generate answer
    answer = generate_pipeline(query)

    # Return retrieved data and the final answer
    return answer


# Build the Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("# 📊 Financial Data QA Bot")
    gr.Markdown(
        "Upload a PDF containing financial data (e.g., P&L tables) and ask questions about the data."
    )

    # File upload
    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])

    # Query input
    query_input = gr.Textbox(label="Enter your financial query", placeholder="e.g., What are the total expenses for Q2 2023?")

    # Output areas
    with gr.Row():
        #retrieved_data_output = gr.Textbox(label="Retrieved Financial Data", lines=10, interactive=False)
        answer_output = gr.Textbox(label="Generated Answer", lines=5, interactive=False)

    # Submit button
    submit_button = gr.Button("Submit")

    # Link the interface with the function
    submit_button.click(
        fn=qa_bot_interface,
        inputs=[pdf_input, query_input],
        outputs=[answer_output],
    )

# Launch the interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f92ac6ffa1c66b0cd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
